In [ ]:
!pip install --upgrade --force-reinstall numpy gensim


  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
Using cached wrapt-1.17.2-cp311-cp

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string
import os

# Initialize NLTK stopwords
stop_words = set(stopwords.words('english'))

from tabulate import tabulate
import numpy as np
import pandas as pd
from scipy.spatial import distance
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from scipy import spatial
import string
import gensim.downloader as api
import string
import string
import seaborn as sns
sns.set_theme()

# importing packages and data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Function to preprocess text
def preprocess(s):
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    s = s.translate(translator)
    # Remove numbers
    translator = str.maketrans('', '', '0123456789')
    s = s.translate(translator)
    # Tokenize text
    tokens = word_tokenize(s)
    # Remove stop words and convert tokens to lowercase
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    filtered_tokens = [stem for stem, token in zip(stemmed_tokens, tokens) if stem != token]
    return ' '.join(filtered_tokens)

# Function to read and preprocess a file
def read_and_preprocess(file_path):
    # Read the file
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read().replace('\n', ' ')
    # Preprocess the text
    preprocessed_text = preprocess(text)
    # Get the filename and construct the new filename for the preprocessed text
    filename = os.path.basename(file_path)
    new_filename = "data_PreP/PreP_" + filename
    # Write the preprocessed text to a new file
    with open(new_filename, 'w', encoding='utf-8') as new_file:
        new_file.write(preprocessed_text)
    return preprocessed_text

# List of file paths to process
file_paths = [
    "A_Furguson_EHCS.txt", "A_Furguson_HPOM.txt",
    "A_Smith_MORA.txt", "A_Smith_WEAL.txt",
    "A_Carlyle_ADAC.txt", "A_Carlyle_AMER.txt",
    "A_Geddes_TSPW.txt", "A_Geddes_LTRD.txt",
    "A_Grant_FNYG.txt", "A_Grant_PFPC.txt",
    "D_Hume_ECHU.txt", "D_Hume_ECPM.txt", "D_Hume_RELI.txt", "D_Hume_TRHN.txt",
    "G_Elliot_PWCE.txt", "G_Elliot_TOFC.txt",
    "H_Mackenzie_JADR.txt", "H_Mackenzie_MANO.txt",
    "H_Mackenzie_MOTW.txt", "H_Mackenzie_OTAF.txt",
    "H_Blair_CDPO.txt", "H_Blair_SERM.txt", "H_Blair_RBLE.txt",
    "J_Adams_POEL.txt", "J_Adams_ASPS.txt",
    "J_Beattie_MINS.txt", "J_Beattie_TEOT.txt", "J_Beattie_EOPM.txt",
    "J_Burnet_OOPL.txt", "J_Burnett_ATMP.txt",
    "J_Craig_PICE.txt",
    "J_Macpherson_FAPH.txt", "J_Macpherson_FING.txt", "J_Macpherson_TEMO.txt", "J_Macpherson_HOGB.txt",
    "J_Thomson_ALFR.txt", "J_Thomson_SEAS.txt", "J_Thomson_LIBE.txt",
    "J_Home_DOUG.txt", "J_Home_AGIS.txt", "J_Home_TSOA.txt",
    "J_Sinclair_OOSD.txt", "J_Sinclair_SAOS.txt", "J_Sinclair_COHL.txt",
    "M_Laing_HOSU1.txt",
    "R_Fergusson_POEM.txt",
    "S_Johnson_AJWI.txt", "S_Johnson_RASS.txt", "S_Johnson_LOTP.txt",
    "T_Sheridan_BEDU.txt", "T_Sheridan_ACOL.txt",
    "W_Shaw_EAPO.txt", "W_Shaw_AAGL.txt"
]

# Dictionary to store preprocessed texts
preprocessed_texts = {}

# Process each file and save preprocessed text
for file_path in file_paths:
    file_path = "data/" + file_path
    preprocessed_texts[file_path] = read_and_preprocess(file_path)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model = api.load("glove-wiki-gigaword-50") #choose from multiple models https://github.com/RaRe-Technologies/gensim-data

In [ ]:
# List of file paths to process
preprocessed_texts = [
        "PreP_A_Furguson_EHCS.txt", "PreP_A_Furguson_HPOM.txt",
    "PreP_A_Smith_MORA.txt", "PreP_A_Smith_WEAL.txt",
    "PreP_A_Carlyle_ADAC.txt", "PreP_A_Carlyle_AMER.txt",
    "PreP_A_Geddes_TSPW.txt", "PreP_A_Geddes_LTRD.txt",
    "PreP_A_Grant_FNYG.txt", "PreP_A_Grant_PFPC.txt",
    "PreP_D_Hume_ECHU.txt", "PreP_D_Hume_ECPM.txt", "PreP_D_Hume_RELI.txt", "PreP_D_Hume_TRHN.txt",
    "PreP_G_Elliot_PWCE.txt", "PreP_G_Elliot_TOFC.txt",
    "PreP_H_Mackenzie_JADR.txt", "PreP_H_Mackenzie_MANO.txt",
    "PreP_H_Mackenzie_MOTW.txt", "PreP_H_Mackenzie_OTAF.txt",
    "PreP_H_Blair_CDPO.txt", "PreP_H_Blair_SERM.txt", "PreP_H_Blair_RBLE.txt",
    "PreP_J_Adams_POEL.txt", "PreP_J_Adams_ASPS.txt",
    "PreP_J_Beattie_MINS.txt", "PreP_J_Beattie_TEOT.txt", "PreP_J_Beattie_EOPM.txt",
    "PreP_J_Burnet_OOPL.txt", "PreP_J_Burnett_ATMP.txt",
    "PreP_J_Craig_PICE.txt",
    "PreP_J_Macpherson_FAPH.txt", "PreP_J_Macpherson_FING.txt", "PreP_J_Macpherson_TEMO.txt", "PreP_J_Macpherson_HOGB.txt",
    "PreP_J_Thomson_ALFR.txt", "PreP_J_Thomson_SEAS.txt", "PreP_J_Thomson_LIBE.txt",
    "PreP_J_Home_DOUG.txt", "PreP_J_Home_AGIS.txt", "PreP_J_Home_TSOA.txt",
    "PreP_J_Sinclair_OOSD.txt", "PreP_J_Sinclair_SAOS.txt", "PreP_J_Sinclair_COHL.txt",
    "PreP_M_Laing_HOSU1.txt",
    "PreP_R_Fergusson_POEM.txt",
    "PreP_S_Johnson_AJWI.txt", "PreP_S_Johnson_RASS.txt", "PreP_S_Johnson_LOTP.txt",
    "PreP_T_Sheridan_BEDU.txt", "PreP_T_Sheridan_ACOL.txt",
    "PreP_W_Shaw_EAPO.txt", "PreP_W_Shaw_AAGL.txt"
]


#print(len(preprocessed_texts))


# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the corpus
corpus = []

# Append preprocessed texts to the corpus
for preprocessed_text in preprocessed_texts:
    # Read the file
    with open("data_PreP/" + preprocessed_text, 'r', encoding='utf-8', errors='ignore') as file:
      text = file.read().replace('\n', ' ')
    corpus.append(text)

# Compute TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

In [ ]:
# Get vectors using the pre-trained GloVe model
def get_vector(s):
    tokens = word_tokenize(s)
    vectors = []
    for token in tokens:
        if token in model.key_to_index:
            vectors.append(model[token])
    if vectors:
        return np.sum(np.array(vectors), axis=0)
    else:
        return np.zeros_like(model["hello"])  # Return a zero vector if no valid word vectors are found

# Cosine similarity with normalized vectors
def cosine_similarity(vec1, vec2):
    vec1_normalized = vec1 / np.linalg.norm(vec1)
    vec2_normalized = vec2 / np.linalg.norm(vec2)
    return np.dot(vec1_normalized, vec2_normalized)

# Calculate similarities
results = []
for i in range(len(corpus)):
  for j in range(i, len(corpus)) :
    sim = cosine_similarity(get_vector(corpus[i]), get_vector(corpus[j]))
    results.append([preprocessed_texts[i], preprocessed_texts[j], sim])

# Sort results based on similarity coefficients in descending order
results.sort(key=lambda x: x[2], reverse=True)

# Print table
print(tabulate(results, headers=["Text File 1", "Text File 2", "Cosine Similarity"]))